In [ ]:
import json
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import folium
import ipinfo
import numpy as np
from datetime import datetime
from zipfile import ZipFile

# LibreSpeed

In [ ]:
folder = os.path.join("labo_tests", "LS")
client_folder = os.path.join(folder, "salameche", "output")
server_folder = os.path.join(folder, "bulbizarre")

In [ ]:
data = []

for folder in os.listdir(client_folder):
    c_dir = os.path.join(client_folder, folder)
    bw, lat = folder.split("_")
    bw = int(bw.rstrip("mbit"))
    lat = int(lat.rstrip("ms"))
    token = None
    result = None
    with open(os.path.join(c_dir, "token")) as f:
        token = f.read().strip("\n")
    with open(os.path.join(server_folder, token, "test.json")) as res:
        result = json.load(res)
    # print(f"Test with {bw}Mbps and {lat}ms")
    # print(f"{bw} got {result['dlStatus']} {result['ulStatus']}")
    # print(f"{2*lat} got {result['pingStatus']}\n")
    data.append({
        "expected_bw" : bw,
        "dl_bw": float(result['dlStatus']),
        "ul_bw": float(result['ulStatus']),
        "expected_latency" : 2*lat,
        "real_latency": float(result['pingStatus']),
        "dl_rpm": int(result['rpmDlStatus']),
        "ul_rpm": int(result['rpmUlStatus'])
    })

df = pd.DataFrame(data)

In [ ]:
df.describe()

In [ ]:
df.loc[df["dl_bw"] < 1]

## Compute QUIC `overheadCompensationFactor`

In [ ]:
factors = []

for _, row in df.iterrows():
    if row["expected_bw"] != 1: # avoid bw test with 0Mbps
        factors.append(row["expected_bw"]/row["dl_bw"])
        factors.append(row["expected_bw"]/row["ul_bw"])

In [ ]:
factor = np.median(factors)

print(f"median : {factor}, var : {np.var(factors)}")

In [ ]:
df_corr = pd.DataFrame({
    "exp_bw" : df["expected_bw"],
    "dl_corr": df.apply(lambda r: factor*r["dl_bw"], axis=1),
    "ul_corr": df.apply(lambda r: factor*r["ul_bw"], axis=1),
    "expected_latency" : df["expected_latency"]
})

In [ ]:
ratios = []

for _, row in df_corr.iterrows():
    if row["exp_bw"] != 1:
        ratios.append(row["exp_bw"]/row["dl_corr"])
        ratios.append(row["exp_bw"]/row["ul_corr"])

np.mean(ratios), np.var(ratios)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10, 4))

ax1.plot([1, 100], [1, 100], color="red")
ax2.plot([1, 100], [1, 100], color="red", label="expeted")

ax1.scatter(df["expected_bw"], df["dl_bw"], c=df["expected_latency"], cmap='viridis')
sca = ax2.scatter(df["expected_bw"], df["ul_bw"], c=df["expected_latency"], cmap='viridis', label="measured")

cbar = plt.colorbar(sca, ax=ax2)
cbar.set_label("Fixed RTT (ms)")

# fig.suptitle(f"Comparaison between expected and measured bandwidth")
fig.supxlabel("Fixed bandwidth (Mbps)")
ax1.set_ylabel("Measured bandwidth (Mbps)")
ax1.set_title("download")
ax2.set_title("upload")
ax1.grid()
ax2.grid()

plt.legend()
# ax1.set_xscale("log")
# ax2.set_xscale("log")

plt.subplots_adjust(
    wspace=0.02,
    hspace=0.2,
    top=0.85,
    bottom=0.15
)
plt.savefig("output/LS_lab/bandwidth.pdf")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10, 4))

ax1.plot([1, 100], [1, 100], color="red")
ax2.plot([1, 100], [1, 100], color="red", label="expeted")

ax1.scatter(df_corr["exp_bw"], df_corr["dl_corr"],
            c=df_corr["expected_latency"], cmap='viridis')
sca = ax2.scatter(df_corr["exp_bw"], df_corr["ul_corr"],
                  c=df_corr["expected_latency"], cmap='viridis', label="measured")

cbar = plt.colorbar(sca, ax=ax2)
cbar.set_label("Fixed RTT (ms)")

# fig.suptitle(f"Comparaison between expected and measured bandwidth after correction")
fig.supxlabel("Fixed bandwidth (Mbps)")
ax1.set_ylabel("Measured bandwidth (Mbps)")
ax1.set_title("download")
ax2.set_title("upload")
ax1.grid()
ax2.grid()

plt.legend()
# ax1.set_xscale("log")
# ax2.set_xscale("log")

plt.subplots_adjust(
    wspace=0.02,
    hspace=0.2,
    top=0.85,
    bottom=0.15
)
plt.savefig("output/LS_lab/corr_bandwidth.pdf")


## Latency

In [ ]:
factor_ratio = []
factor_delta = []

for _, row in df.iterrows():
    factor_ratio.append(row["real_latency"]/row["expected_latency"])
    factor_delta.append(row["real_latency"] - row["expected_latency"])

print(f"mean : {np.median(factor_ratio)}\nvar  : {np.var(factor_ratio)}")
print(f"mean : {np.median(factor_delta)}\nvar  : {np.var(factor_delta)}")


In [ ]:
plt.plot([5, 250], [5, 250], color="red", label="expected")
plt.scatter(df["real_latency"], df["expected_latency"], label="measured")
plt.xlabel("Fixed RTT (ms)")
plt.ylabel("Measured RTT (ms)")
plt.legend()
# plt.title("Comparaison between fixed and measured latency")
plt.savefig("output/LS_lab/latency.pdf")


In [ ]:
rpm_df = pd.DataFrame({
    "real_bw": df["expected_bw"],
    "real_lat": df["expected_latency"],
    "ping_as_rpm": df.apply(lambda r: int(60_000/r["real_latency"]), axis=1),
    "dl_rpm": df["dl_rpm"],
    "dl_rpm_ratio": df.apply(lambda r: (60_000/r["real_latency"])/r["dl_rpm"], axis=1),
    "ul_rpm": df["ul_rpm"],
    "ul_rpm_ratio": df.apply(lambda r: (60_000/r["real_latency"])/r["ul_rpm"], axis=1),
})

In [ ]:
rpm_df

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10, 4))
ax1.scatter(rpm_df["real_bw"], rpm_df["dl_rpm_ratio"], c=rpm_df["real_lat"], cmap='viridis')
sca = ax2.scatter(rpm_df["real_bw"], rpm_df["ul_rpm_ratio"], c=rpm_df["real_lat"], cmap='viridis')
cbar = plt.colorbar(sca, ax=ax2)
cbar.set_label("Fixed RTT (ms)")
# fig.suptitle(f"Factor of latency increase with bandwidth")
fig.supxlabel("Bandwidth (Mbps)")
ax1.set_ylabel("Factor of latency increase")
ax1.set_title("download")
ax2.set_title("upload")
ax1.grid()
ax2.grid()

# ax1.set_xscale("log")
# ax2.set_xscale("log")

plt.subplots_adjust(
    wspace=0.02,
    hspace=0.2,
    top=0.85,
    bottom=0.15
)
plt.savefig("output/LS_lab/factor_latency.pdf")


In [ ]:
rpm_df.loc[(rpm_df["dl_rpm_ratio"] > 20) | (rpm_df["ul_rpm_ratio"] > 20)]

In [ ]:
rpm_df["max_bytes_in_flight"] = rpm_df.apply(lambda r:8*r["real_bw"]*r["real_lat"]*1000/1000_000, axis=1)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10, 4))

ax1.scatter(rpm_df["max_bytes_in_flight"], rpm_df["dl_rpm_ratio"])
ax2.scatter(rpm_df["max_bytes_in_flight"], rpm_df["ul_rpm_ratio"])

# fig.suptitle(f"Factor of latency increase with bandwidth")
fig.supxlabel("Bandwidth delay product (kB)")
ax1.set_ylabel("Factor of latency increase")
ax1.set_title("download")
ax2.set_title("upload")
ax1.grid()
ax2.grid()

plt.subplots_adjust(
    wspace=0.02,
    hspace=0.2,
    top=0.85,
    bottom=0.15
)
plt.savefig("output/LS_lab/BDP.pdf")

## QUIC utilisation

In [ ]:
def parse(filename):
    output = []
    with open(filename, "r") as file:
        for line in file.readlines():
            try:
                output.append(json.loads(line))
            except:
                pass
    return output

In [ ]:
quic_usage = []
http_2_percentage_of_connection = []
http_3_percentage_of_connection = []

for folder in os.listdir(client_folder):
    c_dir = os.path.join(client_folder, folder)
    bw, lat = folder.split("_")
    bw = int(bw.rstrip("mbit"))
    lat = int(lat.rstrip("ms"))
    token = None
    result = None
    with open(os.path.join(c_dir, "token")) as f:
        token = f.read().strip("\n")
    logfile = os.path.join(server_folder, token, "proxy.log")
    if os.path.exists(logfile):
        try:
            proxy_logs = parse(logfile)
            if len(proxy_logs) > 0:
                n_req = h3 = 0
                for i, access in enumerate(proxy_logs):
                    if access["request"]["method"] == "GET" or access["request"]["method"] == "POST":
                        if access["request"]["proto"] == "HTTP/3.0":
                            h3 += 1
                            http_3_percentage_of_connection.append(
                                100*(i+1)/len(proxy_logs))
                        else:
                            http_2_percentage_of_connection.append(100*(i+1)/len(proxy_logs))
                        n_req += 1
                quic_usage.append(100*h3/n_req)
        except:
            pass

In [ ]:
def get_cdf_data(values):
    sorted_values = np.sort(values)
    cdf = np.arange(len(sorted_values)) / len(sorted_values)
    return sorted_values, cdf

In [ ]:
d, c = get_cdf_data(quic_usage)
plt.step(d, c)
plt.xlabel("Percentage of HTTP/3 utilization (%)")
plt.ylabel("Cumulative Probability")
# plt.xlim(xmin=0, xmax=100)
plt.savefig("output/LS_lab/quic_usage.pdf")

In [ ]:
d, c = get_cdf_data(http_2_percentage_of_connection)
plt.hist(http_3_percentage_of_connection, label="HTTP/3")
plt.hist(http_2_percentage_of_connection, label="HTTP/2")
plt.legend()
xlab = plt.xlabel("Occurence of HTTP request according\nto the progress of the connection (in percentage)")
plt.subplots_adjust(wspace=0.04, hspace=0.4, top=0.8, bottom=0.12)
plt.savefig("output/LS_lab/http2_events.pdf", bbox_extra_artists=(xlab, ))